In [ ]:


import cv2
import numpy as np
import os
import glob # For finding calibration images if used instead of video


CHESSBOARD_CORNERS_ROWCOL = (7, 7) # Detect a 7x7 pattern
CHESSBOARD_SQUARE_SIZE_MM = 20.0  # Size of a chessboard square in millimeters


FULL_BOARD_INNER_CORNERS_ROWCOL = (20, 20) # Assuming a 21x21 square physical board

POSTPROCESS_BASE_DIR = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess"
os.makedirs(POSTPROCESS_BASE_DIR, exist_ok=True) # Ensure the directory exists

# Path to the video file containing the chessboard pattern for camera calibration.
calibration_path = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\Resultater\Sjakkbrett_cali1.avi"
# File to save/load camera calibration data (camera matrix, distortion coefficients, mm_per_pixel).
calibration_data_file = os.path.join(POSTPROCESS_BASE_DIR, "calibration_data_4.npz")


offset_x = FULL_BOARD_INNER_CORNERS_ROWCOL[0] - CHESSBOARD_CORNERS_ROWCOL[0]

# offset_y: The starting row index for the top-right 7x7 section.
# This is 0, as it's the "top" row.
offset_y = 0
offset = np.array([offset_x, offset_y], dtype=np.float32)

# objp now represents the 3D coordinates of the sub-pattern's corners
# relative to the full chessboard's origin (e.g., its top-left inner corner).
objp = np.zeros((CHESSBOARD_CORNERS_ROWCOL[0] * CHESSBOARD_CORNERS_ROWCOL[1], 3), np.float32)
# Create a grid for the sub-pattern (from 0,0 to CHESSBOARD_CORNERS_ROWCOL-1)
sub_pattern_grid = np.mgrid[0:CHESSBOARD_CORNERS_ROWCOL[0], 0:CHESSBOARD_CORNERS_ROWCOL[1]].T.reshape(-1,2)
# Apply the offset and scale to real-world coordinates
objp[:,:2] = (sub_pattern_grid + offset) * CHESSBOARD_SQUARE_SIZE_MM


objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# --- Load or Perform Calibration ---
mtx, dist, mm_per_pixel = None, None, None

if os.path.exists(calibration_data_file):
    print(f"Loading calibration data from {calibration_data_file}")
    data = np.load(calibration_data_file, allow_pickle=True) # Added allow_pickle=True
    mtx = data['mtx']
    dist = data['dist']
    if 'mm_per_pixel' in data:
        mm_per_pixel = data['mm_per_pixel']
    else:
        print("mm_per_pixel not found in calibration data, will attempt to calculate.")
else:
    print(f"Calibration data file not found. Performing calibration using video: {calibration_path}")
    cap_cal = cv2.VideoCapture(calibration_path)
    if not cap_cal.isOpened():
        raise IOError(f"Cannot open calibration video file at {calibration_path}")

    frames_for_calibration_count = 0
    max_frames_to_process = 50 # Limit number of frames to check for chessboard - Increased from 100
    processed_frames = 0

    print("Processing calibration video frames...")
    while cap_cal.isOpened() and processed_frames < max_frames_to_process:
        ret_cal, frame_cal = cap_cal.read()
        processed_frames += 1
        if not ret_cal:
            break

        gray_cal = cv2.cvtColor(frame_cal, cv2.COLOR_BGR2GRAY)
        
  
      
        # cv2.CALIB_CB_ADAPTIVE_THRESH: Use adaptive thresholding to convert the image to black and white, rather than a fixed threshold level.
        # cv2.CALIB_CB_NORMALIZE_IMAGE: Normalize the image gamma with equalizeHist before applying fixed or adaptive thresholding.
        # cv2.CALIB_CB_FAST_CHECK: Run a fast check on the image that looks for chessboard corners, and shortcut the call if none is found.
        #                        This can dramatically speed up the call in the degenerate condition when no chessboard is observed.
        # cv2.CALIB_CB_EXHAUSTIVE: Run exhaustive search for chessboard corners. Useful when conditions are challenging. (Can be slow)
        # cv2.CALIB_CB_ACCURACY: Increase the accuracy of corner refinement.
        
        # For findChessboardCornersSB, CALIB_CB_NORMALIZE_IMAGE is a common flag.
        # CALIB_CB_ADAPTIVE_THRESH is not typically used with findChessboardCornersSB.
        finder_flags_sb = cv2.CALIB_CB_NORMALIZE_IMAGE
        # For more challenging conditions, consider adding cv2.CALIB_CB_EXHAUSTIVE:
        # finder_flags_sb = cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_EXHAUSTIVE
        # Or even just cv2.CALIB_CB_EXHAUSTIVE if NORMALIZE_IMAGE causes issues or isn't needed.
        # finder_flags_sb = cv2.CALIB_CB_EXHAUSTIVE

        ret_corners, corners = cv2.findChessboardCornersSB(gray_cal, CHESSBOARD_CORNERS_ROWCOL, flags=finder_flags_sb)

        # If found, add object points, image points (after refining them)
        if ret_corners == True:
            objpoints.append(objp)
            
            # Refine corner locations - cornerSubPix is still useful even with findChessboardCornersSB
            corners2 = cv2.cornerSubPix(gray_cal, corners, (11,11), (-1,-1), 
                                        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            imgpoints.append(corners2)
            frames_for_calibration_count += 1

            # Draw and display the corners (optional, but very useful for debugging)
            cv2.drawChessboardCorners(frame_cal, CHESSBOARD_CORNERS_ROWCOL, corners2, ret_corners)
            cv2.imshow('Calibration Frame', frame_cal)
            if cv2.waitKey(50) & 0xFF == ord('q'): # Display for 50ms, press 'q' to quit this loop early
                 break
            print(f"Found corners in frame {processed_frames}, total frames with corners: {frames_for_calibration_count}")
        else:
            if processed_frames % 10 == 0:
                 print(f"Processed {processed_frames} frames, corners not found in current frame.")
            # Display frame even if corners not found, to see what the camera sees
            cv2.imshow('Calibration Frame', frame_cal)
            if cv2.waitKey(1) & 0xFF == ord('q'): # Minimal wait
                 break

            # Save the first few frames where corners are not found for debugging
            if processed_frames <= 5: # Save first 5 processed frames if corners not found
                debug_filename = f"debug_calibration_frame_{processed_frames:03d}.png"
                # Ensure the PostProcess directory exists or specify a full path where you have write access
                debug_save_path = os.path.join(POSTPROCESS_BASE_DIR, debug_filename)
                try:
                    cv2.imwrite(debug_save_path, frame_cal) # Save the color frame
                    print(f"Saved {debug_save_path} for inspection.")
                except Exception as e:
                    print(f"Error saving debug frame {debug_save_path}: {e}")


    cap_cal.release()
    cv2.destroyWindow('Calibration Frame') # if imshow was used

    if len(objpoints) > 0 and len(imgpoints) > 0:
        print(f"\nPerforming camera calibration with {len(objpoints)} images...")
        rms_error, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_cal.shape[::-1], None, None)
        
        print(f"RMS re-projection error from calibrateCamera: {rms_error:.4f}")

        if rms_error >= 1.0: # Check if RMS error is high
            print(f"WARNING: High RMS re-projection error: {rms_error:.4f}. Calibration results (mtx, dist) may be inaccurate.")
            print("This can significantly affect the accuracy of undistortion and any measurements derived from it.")
            print("Suggestions to improve RMS error:")
            print("  1. Ensure CHESSBOARD_SQUARE_SIZE_MM is accurate for your physical board.")
            print("  2. Use a diverse set of high-quality calibration frames: vary the chessboard's position, orientation (tilt, rotation), and distance from the camera. Ensure the board is flat and well-lit.")
            print("  3. CRITICAL CHECK: Ensure 'FULL_BOARD_INNER_CORNERS_ROWCOL' accurately reflects your physical board's total inner corners.")
            print(f"     Current CHESSBOARD_CORNERS_ROWCOL (detected pattern): {CHESSBOARD_CORNERS_ROWCOL}")
            print(f"     Current FULL_BOARD_INNER_CORNERS_ROWCOL (physical board): {FULL_BOARD_INNER_CORNERS_ROWCOL}")
            print(f"     The script is configured to map the detected {CHESSBOARD_CORNERS_ROWCOL} pattern to the top-right section of the {FULL_BOARD_INNER_CORNERS_ROWCOL} physical board.")
            print("     If this mapping, the physical board size, or the detected pattern visibility is incorrect, it can lead to high RMS error.")
            print("     For instance, if your physical board is smaller (e.g., exactly 7x7 inner corners), then FULL_BOARD_INNER_CORNERS_ROWCOL should be (7,7), and the offset logic would change (offset_x=0, offset_y=0).")
            print("Proceeding with potentially inaccurate calibration parameters...")
            # The script will continue using the mtx and dist obtained, despite the high error.
        else: # rms_error < 1.0
            print(f"Camera calibrated successfully. RMS re-projection error: {rms_error:.4f}")
            
        # Calculate re-projection error manually for each image (good for detailed insight)
        mean_error_manual = 0
        for i in range(len(objpoints)):
            imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
            error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
            mean_error_manual += error
        if len(objpoints) > 0: # Avoid division by zero
            print( "Mean re-projection error (calculated manually per image): {}".format(mean_error_manual/len(objpoints)) )
            
    else:
        print("Not enough points for calibration. Check chessboard parameters and video content.")
        # mtx, dist remain None if this path is taken.

# --- Calculate mm_per_pixel using a frame (if not loaded or needs recalculation) ---
# This section allows manual selection of a frame for mm_per_pixel calculation.
if mtx is not None and dist is not None and (mm_per_pixel is None or mm_per_pixel == 0): # Recalculate if not loaded or zero
    print("\n--- Manual Frame Selection for mm_per_pixel Calculation ---")
    print("Press 's' to select the current frame for mm/pixel calculation.")
    print("Press 'n' or SPACE to view the next frame.")
    print("Press 'q' to quit selection without calculating mm/pixel.")
    
    cap_cal_px = cv2.VideoCapture(calibration_path)
    if not cap_cal_px.isOpened():
        print(f"Could not reopen calibration video for mm_per_pixel calculation: {calibration_path}")
    else:
        found_frame_for_mm_pixel = False
        temp_processed_frames = 0
        
        while cap_cal_px.isOpened():
            ret_px, frame_px = cap_cal_px.read()
            temp_processed_frames += 1
            if not ret_px:
                print("End of calibration video reached during mm/pixel frame selection.")
                break

            h_px, w_px = frame_px.shape[:2]
            # Undistort the frame
            newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w_px,h_px), 1, (w_px,h_px))
            dst_frame_px = cv2.undistort(frame_px, mtx, dist, None, newcameramtx)

            display_frame_px = dst_frame_px.copy() # Work on a copy for drawing
            
            gray_px_undistorted = cv2.cvtColor(dst_frame_px, cv2.COLOR_BGR2GRAY)
            
            finder_flags_sb_px = cv2.CALIB_CB_NORMALIZE_IMAGE
            ret_corners_px, corners_px = cv2.findChessboardCornersSB(gray_px_undistorted, CHESSBOARD_CORNERS_ROWCOL, flags=finder_flags_sb_px)

            if ret_corners_px:
                corners_px_refined = cv2.cornerSubPix(gray_px_undistorted, corners_px, (11,11), (-1,-1),
                                                    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
                # Draw corners on the display frame
                cv2.drawChessboardCorners(display_frame_px, CHESSBOARD_CORNERS_ROWCOL, corners_px_refined, ret_corners_px)
                
            cv2.imshow('Select Frame for mm_per_pixel', display_frame_px)
            key = cv2.waitKey(0) & 0xFF # Wait indefinitely for a key press

            if key == ord('s'): # User selects this frame
                if ret_corners_px:
                    print(f"Frame {temp_processed_frames} selected for mm/pixel calculation.")
                    # Proceed with calculation using corners_px_refined from this frame
                    if CHESSBOARD_CORNERS_ROWCOL[0] > 1:
                        pt1_pixel = corners_px_refined[0][0]
                        pt2_pixel = corners_px_refined[CHESSBOARD_CORNERS_ROWCOL[0]-1][0]
                        pixel_distance = np.sqrt((pt1_pixel[0] - pt2_pixel[0])**2 + (pt1_pixel[1] - pt2_pixel[1])**2)
                        real_world_distance_mm = (CHESSBOARD_CORNERS_ROWCOL[0] - 1) * CHESSBOARD_SQUARE_SIZE_MM
                        
                        if pixel_distance > 0:
                            mm_per_pixel = real_world_distance_mm / pixel_distance
                            print(f"Calculated mm_per_pixel: {mm_per_pixel:.4f} (using {CHESSBOARD_CORNERS_ROWCOL[0]-1} squares)")
                            found_frame_for_mm_pixel = True
                            break # Exit the while loop for frame selection
                        else:
                            print("Pixel distance is zero in selected frame. Cannot calculate mm_per_pixel. Try another frame.")
                    else:
                        print("Not enough corners along width (CHESSBOARD_CORNERS_ROWCOL[0] <= 1) to calculate mm_per_pixel.")
                        # Allow user to select another frame or quit
                else:
                    print("Corners not found in this frame. Cannot use for mm/pixel calculation. Press 'n' or SPACE for next, or 'q' to quit.")
            
            elif key == ord('n') or key == ord(' '): # Next frame
                continue
            
            elif key == ord('q'): # Quit selection
                print("mm/pixel calculation aborted by user.")
                break
            else: # Other key pressed
                print("Invalid key. Press 's' to select, 'n' or SPACE for next, 'q' to quit.")


        cap_cal_px.release()
        cv2.destroyWindow('Select Frame for mm_per_pixel') # Clean up the selection window

        if not found_frame_for_mm_pixel:
            print("No suitable frame was selected or found for mm_per_pixel calculation.")
            mm_per_pixel = 0 # Indicate failure or no selection


if os.path.exists(calibration_data_file):
    # If file exists, check if mm_per_pixel was newly calculated and needs saving
    data = np.load(calibration_data_file)
    if 'mm_per_pixel' not in data or (data['mm_per_pixel'] == 0 and mm_per_pixel != 0):
        if mtx is not None and dist is not None: # Ensure mtx and dist are valid
             print(f"Updating calibration data file with new mm_per_pixel: {calibration_data_file}")
             np.savez(calibration_data_file, mtx=mtx, dist=dist, mm_per_pixel=mm_per_pixel)
elif mtx is not None and dist is not None: # File didn't exist, save new calibration
    print(f"Saving new calibration data to: {calibration_data_file}")
    np.savez(calibration_data_file, mtx=mtx, dist=dist, mm_per_pixel=mm_per_pixel)


if mtx is not None:
    print("\n--- Calibration Results ---")
    print("Camera Matrix (mtx):\n", mtx)
    print("\nDistortion Coefficients (dist):\n", dist)
else:
    print("\n--- Calibration Not Performed or Failed ---")

if mm_per_pixel is not None and mm_per_pixel > 0:
    print(f"\nMillimeters per pixel (at waterline, from chessboard): {mm_per_pixel:.4f} mm/pixel")
    print(f"This means 1 pixel = {mm_per_pixel:.4f} mm")
else:
    print("\nMillimeters per pixel could not be determined.")
    print("Ensure chessboard is visible at waterline in calibration video and CHESSBOARD_CORNERS_ROWCOL[0] > 1.")

# Clean up any remaining OpenCV windows if they were used for display
cv2.destroyAllWindows()

Calibration data file not found. Performing calibration using video: C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\Resultater\Sjakkbrett_cali1.avi
Processing calibration video frames...
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_001.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_001.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_002.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_002.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_003.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_003.png for inspection.
Saved C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\debug_calibration_frame_004.png for inspection.
Sa

In [ ]:
#This code takes a video of paper pieces floating in the wake of a ship and tracks them to show the flow of the water.
#It uses OpenCV to process the video and matplotlib to plot the results.
import cv2
import numpy as np  
import os
import re # Import regular expression module

# --- Configuration ---
# Path to the video file to be analyzed (e.g., paper pieces in water flow).
path = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\Resultater\papirtest6 - 21 sek.avi"
#path = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\Resultater\papirtest7 - 34 sek.avi"
mm_per_pixel = 0.42511 #mm
# Background subtractor to isolate paper cutouts
# SUGGESTION 3: Try reducing varThreshold if pieces are faint (e.g., 12 or 10)
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=12, detectShadows=False) # Original: varThreshold=16
# SUGGESTION 4: If MORPH_OPEN kernel (5,5) is too large for small pieces, try (3,3)
kernel = np.ones((5,5), np.uint8) # Original: (5,5)
dilation_kernel = np.ones((3,3), np.uint8) # Kernel for dilation

# Lucas–Kanade optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.2, minDistance=15, blockSize=7)
lk_params = dict(winSize=(15,15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

MAX_POINT_AGE_FRAMES = 190 # Max age for a point in frames (e.g., 2.5 seconds at 60 fps)

# --- Video setup ---
cap = cv2.VideoCapture(path)
if not cap.isOpened():
    raise IOError(f"Cannot open video file at {path}")

# Read first frame for initial setup and main loop
ret, old_frame = cap.read()
if not ret or old_frame is None:
    raise IOError("Failed to read first frame from video.")
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
h, w = old_gray.shape[:2]




# test_threshold_value = 145  # START WITH THIS VALUE AND ADJUST (0-255)
# seconds_to_skip_for_test = 2.0
# frame_for_test = None
# gray_for_test = None
# 
# fps_test = cap.get(cv2.CAP_PROP_FPS)
# if fps_test > 0:
#     target_frame_idx_test = int(seconds_to_skip_for_test * fps_test)
#     cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # Reset to beginning before reading for test
#     current_frame_count = 0
#     while current_frame_count <= target_frame_idx_test:
#         ret_test, temp_frame = cap.read()
#         if not ret_test:
#             print(f"Warning: Could not reach frame at {seconds_to_skip_for_test}s for testing. Using first frame instead.")
#             # Need to ensure old_frame_initial was read if this path is taken.
#             # For simplicity, if this fails, we'll rely on the first frame read for old_gray.
#             # This section is commented out, so this logic is moot.
#             # frame_for_test = old_frame.copy() # Use the already read first frame
#             break
#         frame_for_test = temp_frame
#         current_frame_count += 1
#     
#     if frame_for_test is not None:
#         gray_for_test = cv2.cvtColor(frame_for_test, cv2.COLOR_BGR2GRAY)
#     else: 
#         print("Warning: frame_for_test is None. Using first frame's grayscale.")
#         gray_for_test = old_gray.copy() # Use the already read first frame
# else:
#     print("Warning: FPS is 0. Using first frame for threshold test.")
#     # frame_for_test = old_frame.copy() # Use the already read first frame
#     gray_for_test = old_gray.copy() # Use the already read first frame's gray
# 
# 
# # Apply thresholding to the chosen frame for visualization
# _, test_thresh_img = cv2.threshold(gray_for_test, test_threshold_value, 255, cv2.THRESH_BINARY)
# 
# # Display the images
# cv2.imshow(f'Original Grayscale (approx {seconds_to_skip_for_test}s)', gray_for_test)
# cv2.imshow(f'Thresholded Output (Value: {test_threshold_value})', test_thresh_img)
# print(f"Displaying threshold test with value: {test_threshold_value} from frame at ~{seconds_to_skip_for_test}s.")
# print("Press any key in an OpenCV window to continue with the main script...")
# cv2.waitKey(0) # Wait for a key press
# cv2.destroyWindow(f'Original Grayscale (approx {seconds_to_skip_for_test}s)')
# cv2.destroyWindow(f'Thresholded Output (Value: {test_threshold_value})')
# print("Threshold test windows closed.")
# 



# Create a mask for static bad regions from the first frame (e.g., the wire)
first_frame_blur = cv2.GaussianBlur(old_gray, (9,9), 2)
# Adjust Canny thresholds if needed to pick up the wire effectively but not too much noise
static_bad_regions_mask = cv2.Canny(first_frame_blur, 50, 150)
# Dilate to make the wire mask thicker, ensure dilation_kernel is defined
static_bad_regions_mask = cv2.dilate(static_bad_regions_mask, dilation_kernel, iterations=2)

# --- Speed Map Configuration ---
GRID_CELL_SIZE = 16 # pixels (adjust as needed)
SAVE_SPEED_MAP_INTERVAL_SECONDS = 5 # Save speed map every 5 seconds

# Dynamically determine speed map save path based on video filename
video_filename_base = os.path.basename(path)
match = re.search(r"papirtest(\d+)", video_filename_base, re.IGNORECASE)

speed_map_folder_name = "SpeedMaps_general" # Default folder name
if match:
    test_number = match.group(1)
    speed_map_folder_name = f"SpeedMaps_papirtest{test_number}"
    print(f"Speed maps will be saved in a folder specific to test number: {test_number}")
else:
    print(f"No 'papirtest<number>' pattern found in filename. Speed maps will be saved in: {speed_map_folder_name}")

# Assuming POSTPROCESS_BASE_DIR is available from the first cell.
# If not, it needs to be defined here or passed appropriately.
# For robustness, let's ensure POSTPROCESS_BASE_DIR is defined or fall back.
try:
    # Check if POSTPROCESS_BASE_DIR is defined and is a string
    if not (isinstance(POSTPROCESS_BASE_DIR, str) and POSTPROCESS_BASE_DIR):
        # Fallback if POSTPROCESS_BASE_DIR is not a valid string
        print("Warning: POSTPROCESS_BASE_DIR from Cell 1 not found or invalid. Using current script's parent directory for PostProcess.")
        # This fallback assumes the script is in a subdirectory of the intended PostProcess base.
        # A more robust fallback might be a hardcoded path or raising an error.
        # For this example, let's use a path relative to the script's location if PostProcess is not defined.
        # A simple approach if POSTPROCESS_BASE_DIR is not found:
        _default_postprocess_base = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess"
        print(f"Using default POSTPROCESS_BASE_DIR: {_default_postprocess_base}")
        speed_map_save_path = os.path.join(_default_postprocess_base, speed_map_folder_name)
    else:
        speed_map_save_path = os.path.join(POSTPROCESS_BASE_DIR, speed_map_folder_name)
except NameError:
    # Fallback if POSTPROCESS_BASE_DIR is not defined at all
    print("Warning: POSTPROCESS_BASE_DIR from Cell 1 is not defined. Using a default base path for PostProcess.")
    _default_postprocess_base = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess" # Define a default
    print(f"Using default POSTPROCESS_BASE_DIR: {_default_postprocess_base}")
    speed_map_save_path = os.path.join(_default_postprocess_base, speed_map_folder_name)


# speed_map_save_path = r"C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps" # Ensure this folder exists
if not os.path.exists(speed_map_save_path):
    os.makedirs(speed_map_save_path)
    print(f"Created speed map directory: {speed_map_save_path}")
else:
    print(f"Speed map directory already exists: {speed_map_save_path}")

grid_cols = int(np.ceil(w / GRID_CELL_SIZE))
grid_rows = int(np.ceil(h / GRID_CELL_SIZE))

speed_sum_grid = np.zeros((grid_rows, grid_cols), dtype=np.float32)
speed_count_grid = np.zeros((grid_rows, grid_cols), dtype=np.int32)
# --- End Speed Map Configuration ---

# --- Video Trimming Configuration ---
TRIM_VIDEO = True # Set to True to enable skipping a section
# Times are in seconds from the start of the video.
# The section from TRIM_SKIP_START_SECONDS up to (but not including) TRIM_RESUME_AT_SECONDS will be skipped.
TRIM_SKIP_START_SECONDS = 10.5  # Updated: Start skipping at 11 seconds
TRIM_RESUME_AT_SECONDS = 14.7 # Updated: Resume processing AT 16 seconds (frame at 16s will be processed)

skip_from_frame_0idx = -1
resume_at_frame_0idx = -1
can_trim_video = False # Internal flag to confirm trimming is possible

# --- End Video Trimming Configuration ---


# Define the new pentagonal ROI
x_min_roi = w // 4
x_max_roi = 3 * w // 4

# Calculate y-coordinate for the intersection on the left slope
# Original left slope: from (0,0) to (w//2, h//2)
# Line equation: y = (h/w) * x
# At x = x_min_roi:
y_intersect_left = 0
if w > 0: # Avoid division by zero if w is 0
    y_intersect_left = (h * x_min_roi) // w
y_intersect_left = max(0, min(y_intersect_left, h // 2)) # Ensure it's within triangle bounds

# Calculate y-coordinate for the intersection on the right slope
# Original right slope: from (w-1,0) to (w//2, h//2)
V2_x_float = float(w - 1)
V2_y_float = 0.0
V3_x_float = float(w // 2)
V3_y_float = float(h // 2)

y_intersect_right = 0
den_slope_right = V3_x_float - V2_x_float
if abs(den_slope_right) < 1e-6: # Handles case where V3_x is same as V2_x (e.g. w=2)
    if x_max_roi == int(V2_x_float): # If x_max_roi is on the vertical line
         y_intersect_right = h // 2 # Max y along that segment up to tip
    else:
         y_intersect_right = 0 # Otherwise, no intersection or point is at y=0
else:
    slope_right = (V3_y_float - V2_y_float) / den_slope_right
    # y = V2_y + slope_right * (x_max_roi - V2_x)
    y_intersect_right = int(round(V2_y_float + slope_right * (float(x_max_roi) - V2_x_float)))
y_intersect_right = max(0, min(y_intersect_right, h // 2)) # Ensure it's within triangle bounds


# Note: Variable name 'triangle_roi' is kept for minimal changes,
# but it now defines a pentagon.
triangle_roi = np.array([
    [x_min_roi, 0],                # Top-left of restricted strip
    [x_max_roi, 0],                # Top-right of restricted strip
    [x_max_roi, y_intersect_right],# Point on the right slope
    [w // 2, h],              # Original triangle tip (center of top half)
    [x_min_roi, y_intersect_left] # Point on the left slope
], dtype=np.int32)

# Unified and more sensitive HoughCircles parameters
# SUGGESTION 1: Experiment with hough_params
hough_params = dict(
    dp=1.2, 
    minDist=10,  # Circles can be closer. Original: 10. Try 7 or 5 if pieces are close.
    param1=40,   # Original: 60. Try 50 or 40 if param2 adjustment isn't enough.
    param2=15,   # Original: 20. Try 15, 12, or 10 to make detection less strict.
    minRadius=5, # Original: 4. Verify this range.
    maxRadius=25 # Original: 25. Verify this range.
)

# --- ADD THRESHOLDING FOR WHITE OBJECTS ---
# Adjust this threshold value (0-255). Higher values are more selective for brighter whites.
# Start with a value like 200 or 220 and adjust based on results.
# If paper is "darker white, gray ish", this value was likely too high. Try a lower value.
# USE THE VALUE YOU FOUND FROM THE TEST SECTION ABOVE
white_threshold_value = 145 # Set to the determined good value.
# --- END THRESHOLDING CONFIG ---

# Initial circle detection in the first frame
# Apply thresholding to isolate white regions
_, old_gray_thresh = cv2.threshold(old_gray, white_threshold_value, 255, cv2.THRESH_BINARY)
# SUGGESTION 2: Experiment with GaussianBlur parameters on the thresholded image
gray_blur = cv2.GaussianBlur(old_gray_thresh, (9,9), 2) # Original: (9,9), 2. Try (5,5), 1.5 or (7,7), 2
circles = cv2.HoughCircles(
    gray_blur, cv2.HOUGH_GRADIENT,
    **hough_params
)

p0_birth_frames = [] # Stores birth frame_idx for each point in p0

if circles is not None:
    all_detected_pts = np.uint16(np.around(circles[0]))[:,:2]
    # Keep only circles whose centers are inside the triangular ROI
    pts_in_roi = []
    for pt_candidate in all_detected_pts:
        if cv2.pointPolygonTest(triangle_roi, tuple(pt_candidate), False) >= 0:
            pts_in_roi.append(pt_candidate)
    
    if pts_in_roi:
        pts = np.array(pts_in_roi, dtype=np.uint16)
        p0 = pts.reshape(-1,1,2).astype(np.float32)
        p0_birth_frames = [0] * len(p0) # Initial points born at frame 0
    else:
        p0 = np.empty((0,1,2), dtype=np.float32)
else:
    p0 = np.empty((0,1,2), dtype=np.float32)

# Mask for drawing tracks
mask = np.zeros_like(old_frame)

# Parameters for dynamic re-detection
detect_interval = 1    # every N frames
min_features    = 5    # if tracking falls below this, re-detect (changed from 0)
frame_idx       = 0 

# Compute display delay
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps) if fps > 0 else 30
save_interval_frames = int(SAVE_SPEED_MAP_INTERVAL_SECONDS * fps) if fps > 0 else -1 # -1 to disable if fps is 0

# Calculate frame numbers for trimming (0-indexed) based on FPS
if TRIM_VIDEO: # Only proceed if user initially wants to trim
    if "papirtest6" in path: # Updated: Check if "papirtest6" is in the filename
        if fps > 0:
            skip_from_frame_0idx = int(TRIM_SKIP_START_SECONDS * fps)
            resume_at_frame_0idx = int(TRIM_RESUME_AT_SECONDS * fps)
            if skip_from_frame_0idx < resume_at_frame_0idx:
                can_trim_video = True
                print(f"Video trimming enabled for '{os.path.basename(path)}': Skipping from frame {skip_from_frame_0idx} (0-indexed) up to {resume_at_frame_0idx - 1}.")
                print(f"Processing will resume at frame {resume_at_frame_0idx} (0-indexed).")
            else:
                print(f"Warning: Trim start time is not before resume time. Trimming disabled for '{os.path.basename(path)}'.")
                TRIM_VIDEO = False 
        else:           # fps <= 0
            print(f"Warning: FPS is 0, cannot calculate trim frames. Trimming disabled for '{os.path.basename(path)}'.")
            TRIM_VIDEO = False
    else:         # "papirtest6" (or the specified pattern) not in path
        print(f"Filename '{os.path.basename(path)}' does not contain 'papirtest6'. Specific trimming disabled.")
        TRIM_VIDEO = False
# If TRIM_VIDEO was initially False, or set to False above, trimming will be disabled

has_performed_skip = False # Flag to ensure skip jump happens only once per defined section

# --- Function to generate and display speed map ---
def display_speed_map(speed_sum, speed_count, frame_to_overlay, cell_size, mm_per_pixel_val, fps_val):
    MIN_SAMPLES_FOR_RELIABLE_MAX = 10 # Min samples in a cell for its max speed to be considered "stable"

    base_display = frame_to_overlay.copy()
    if np.sum(speed_count) == 0:
        cv2.putText(base_display, "No speed data yet", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.imshow('Speed Map', base_display)
        return base_display

    avg_speed_grid_px_per_frame = np.zeros_like(speed_sum)
    valid_cells = speed_count > 0 # Cells with at least one measurement
    avg_speed_grid_px_per_frame[valid_cells] = speed_sum[valid_cells] / speed_count[valid_cells]

    units = "pixels/frame"
    avg_speed_grid_for_colormap = avg_speed_grid_px_per_frame.copy()
    conversion_successful = False

    if mm_per_pixel_val is not None and mm_per_pixel_val > 0 and fps_val is not None and fps_val > 0:
        try:
            avg_speed_mm_per_frame = avg_speed_grid_px_per_frame * float(mm_per_pixel_val)
            avg_speed_mm_per_second = avg_speed_mm_per_frame * float(fps_val)
            avg_speed_m_per_second = avg_speed_mm_per_second / 1000.0
            avg_speed_grid_for_colormap = avg_speed_m_per_second
            units = "m/s"
            conversion_successful = True
        except Exception as e:
            print(f"Error during speed conversion: {e}. Displaying in pixels/frame.")
    else:
        # Check if frame_idx is defined globally, otherwise use a simple modulo for periodic warning
        # This warning is now less critical if mm_per_pixel is not available, as text will indicate it.
        # if 'frame_idx' in globals() and frame_idx % (int(fps_val) if fps_val > 0 else 30) == 0:
        #     print(f"Warning: mm_per_pixel ({mm_per_pixel_val}) or FPS ({fps_val}) is invalid. Speed map shows pixels/frame.")
        pass


    # Determine min/max for colormap normalization (using all valid cells)
    min_for_colormap = 0
    max_for_colormap = 0
    if np.any(valid_cells):
        min_for_colormap = np.min(avg_speed_grid_for_colormap[valid_cells])
        max_for_colormap = np.max(avg_speed_grid_for_colormap[valid_cells])
    
    normalized_speed_grid = np.zeros_like(avg_speed_grid_for_colormap, dtype=np.uint8)
    if max_for_colormap > min_for_colormap: # Avoid division by zero
        normalized_speed_grid[valid_cells] = (
            (avg_speed_grid_for_colormap[valid_cells] - min_for_colormap) / 
            (max_for_colormap - min_for_colormap) * 255
        ).astype(np.uint8)
    elif np.any(valid_cells): # If all speeds are the same (but non-zero count)
         normalized_speed_grid[valid_cells] = 128 # Assign a mid-range color

    colored_speed_map_small = cv2.applyColorMap(normalized_speed_grid, cv2.COLORMAP_JET)

    speed_map_display_overlay = np.zeros_like(frame_to_overlay, dtype=np.uint8)
    for r_idx in range(grid_rows):
        for c_idx in range(grid_cols):
            if speed_count[r_idx, c_idx] > 0: # Only draw for cells with data
                start_x, start_y = c_idx * cell_size, r_idx * cell_size
                end_x, end_y = min(start_x + cell_size, frame_to_overlay.shape[1]), min(start_y + cell_size, frame_to_overlay.shape[0])
                
                color_val = colored_speed_map_small[r_idx, c_idx]
                cv2.rectangle(speed_map_display_overlay, (start_x, start_y), (end_x, end_y), 
                              (int(color_val[0]), int(color_val[1]), int(color_val[2])), -1)
    
    alpha = 0.6 
    beta = 1.0 - alpha
    combined_display = cv2.addWeighted(base_display, beta, speed_map_display_overlay, alpha, 0.0)
    
    # --- Text Display ---
    text_y_offset = 30
    cv2.putText(combined_display, f"Speed ({units}):", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3, cv2.LINE_AA)
    cv2.putText(combined_display, f"Speed ({units}):", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)
    
    # Min speed text (always from overall min)
    min_for_text_display = min_for_colormap
    text_y_offset += 25
    cv2.putText(combined_display, f"Min: {min_for_text_display:.3f}", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 3, cv2.LINE_AA)
    cv2.putText(combined_display, f"Min: {min_for_text_display:.3f}", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    # Max speed text (distinguish between stable and peak)
    max_for_text_display = max_for_colormap # Default to overall max
    max_label_qualifier = ""

    reliable_cells_for_max = valid_cells & (speed_count >= MIN_SAMPLES_FOR_RELIABLE_MAX)
    if np.any(reliable_cells_for_max):
        max_for_text_display = np.max(avg_speed_grid_for_colormap[reliable_cells_for_max])
        max_label_qualifier = "(stable)"
    elif np.any(valid_cells): # Some data exists, but no cell is "reliable" yet
        max_label_qualifier = "(peak)"
    # If no valid_cells, max_for_text_display remains 0, qualifier empty.

    text_y_offset += 25
    if np.any(valid_cells):
        cv2.putText(combined_display, f"Max: {max_for_text_display:.3f} {max_label_qualifier}", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(combined_display, f"Max: {max_for_text_display:.3f} {max_label_qualifier}", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
    else: # No data at all
        cv2.putText(combined_display, "Max: N/A", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(combined_display, "Max: N/A", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)


    if not conversion_successful and (mm_per_pixel_val is None or mm_per_pixel_val <= 0):
        text_y_offset += 25
        cv2.putText(combined_display, "mm/pixel not calibrated", (10, text_y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Speed Map', combined_display)
    return combined_display
# --- End Function ---


# --- Main processing loop ---
while True:
    # --- Video Trimming Skip Logic ---
    if TRIM_VIDEO and can_trim_video and not has_performed_skip:
        current_read_pos_0idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) # 0-indexed position of next frame to be read

        # Check if the reader is about to enter or is within the defined skip zone
        if current_read_pos_0idx >= skip_from_frame_0idx and current_read_pos_0idx < resume_at_frame_0idx:
            print(f"Video reader at frame {current_read_pos_0idx} (0-indexed). Jumping to resume at frame {resume_at_frame_0idx} (0-indexed).")
            cap.set(cv2.CAP_PROP_POS_FRAMES, float(resume_at_frame_0idx))
            has_performed_skip = True # Mark that the jump has been made for this section

            # Adjust user's frame_idx:
            # The next frame read by cap.read() will be 'resume_at_frame_0idx'.
            # After cap.read() and frame_idx += 1, frame_idx should be (resume_at_frame_0idx + 1).
            # So, before the increment, frame_idx must be resume_at_frame_0idx.
            # And before the read, frame_idx (which is from previous iteration) should be set to (resume_at_frame_0idx -1).
            frame_idx = resume_at_frame_0idx -1 
            
            # Reset optical flow points as the sequence is broken
            p0 = np.empty((0,1,2), dtype=np.float32)
            p0_birth_frames = []
            print(f"Optical flow points reset. User frame_idx set to {frame_idx} (will be {resume_at_frame_0idx+1} after read & increment).")
            # The 'old_gray' will be correctly updated from the frame read after the jump.
    # --- End Video Trimming Skip Logic ---

    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1 # This is the 1-based index of the 'frame' just read
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Update background mask each frame
    fgmask = backSub.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    fgmask = cv2.dilate(fgmask, dilation_kernel, iterations=1) # Dilate to make foreground more robust

    # Periodically detect new circles only in the triangular ROI
    if frame_idx % detect_interval == 0 or len(p0) < min_features:
        # Apply thresholding to isolate white regions
        _, frame_gray_thresh = cv2.threshold(frame_gray, white_threshold_value, 255, cv2.THRESH_BINARY)
        # SUGGESTION 2: Experiment with GaussianBlur parameters (same as above) on the thresholded image
        blur = cv2.GaussianBlur(frame_gray_thresh, (9,9), 2) # Original: (9,9), 2. Try (5,5), 1.5 or (7,7), 2
        circles = cv2.HoughCircles(
            blur, cv2.HOUGH_GRADIENT,
            **hough_params # Use unified parameters
        )
        if circles is not None:
            all_pts_re_detected = np.uint16(np.around(circles[0]))[:,:2]
            # Only keep new detections from the triangular ROI
            new_pts_in_roi = []
            for pt_candidate in all_pts_re_detected:
                if cv2.pointPolygonTest(triangle_roi, tuple(pt_candidate), False) >= 0:
                    new_pts_in_roi.append(pt_candidate)

            if new_pts_in_roi: # Check if any points were actually detected in the region
                new_pts_array = np.array(new_pts_in_roi, dtype=np.uint16)
                new_pts = new_pts_array.reshape(-1,1,2).astype(np.float32)
                
                new_birth_frames = [frame_idx] * len(new_pts)
                if p0.size == 0:
                    p0 = new_pts
                    p0_birth_frames = new_birth_frames
                else:
                    p0 = np.concatenate((p0, new_pts), axis=0)
                    p0_birth_frames.extend(new_birth_frames)
                # Optional: Remove duplicate points if re-detection is very frequent
                # p0, unique_indices = np.unique(p0, axis=0, return_index=True)
                # p0_birth_frames = [p0_birth_frames[i] for i in unique_indices]


    # If we have points to track, compute optical flow
    if len(p0) > 0:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is None or st is None:
            # No points tracked in this frame
            p0 = np.empty((0,1,2), dtype=np.float32) # Clear points if flow fails
            p0_birth_frames = []
            old_gray = frame_gray.copy()
            cv2.imshow('Optical Flow Tracking', frame) # Show frame
            if cv2.waitKey(delay) & 0xFF == ord('q'):
                break
            continue

        # Select only successfully tracked points
        st_flat = st.flatten()
        good_new_candidates = p1[st_flat == 1]
        good_old_candidates = p0[st_flat == 1]
        # Filter corresponding birth frames
        good_birth_frames_candidates = [p0_birth_frames[i] for i, status in enumerate(st_flat) if status == 1]

        # Prune points that have moved off-frame, are too old, or are on the background/static bad regions
        
        surviving_good_new = []
        surviving_good_old = []
        surviving_birth_frames = []

        if good_new_candidates.size > 0: # Check if there are any candidates before iterating
            for i_candidate, pt_new in enumerate(good_new_candidates):
                x_coord_int, y_coord_int = pt_new.ravel().astype(int) # Use int for indexing mask
                y_new_float = pt_new.ravel()[1] # Get new y for movement check
                
                pt_old = good_old_candidates[i_candidate]
                y_old_float = pt_old.ravel()[1] # Get old y for movement check

                birth_frame = good_birth_frames_candidates[i_candidate]
                age = frame_idx - birth_frame

                # Check bounds, age, if in foreground, NOT in a static bad region, AND NOT moving upwards
                if (x_coord_int >= 0 and x_coord_int < w and
                    y_coord_int >= 0 and y_coord_int < h and
                    age <= MAX_POINT_AGE_FRAMES and
                    fgmask[y_coord_int, x_coord_int] == 255 and # Must be in MOG2 foreground
                    static_bad_regions_mask[y_coord_int, x_coord_int] == 0 and # Must NOT be on a static bad region
                    # SUGGESTION 5: Relax upward movement constraint for testing
                    y_new_float >= y_old_float-1): # Original: y_new_float >= y_old_float. Try y_new_float >= y_old_float - 1
                    surviving_good_new.append(pt_new)
                    surviving_good_old.append(pt_old) # Use pt_old directly
                    surviving_birth_frames.append(birth_frame)
        
        # Update points based on survivors
        if surviving_good_new:
            good_new = np.array(surviving_good_new).reshape(-1, 1, 2)
            good_old = np.array(surviving_good_old).reshape(-1, 1, 2)
            
            # Draw the tracks for surviving points
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                x_new, y_new = new.ravel().astype(int)
                x_old, y_old = old.ravel().astype(int)
                mask = cv2.line(mask, (x_new, y_new), (x_old, y_old), (0,255,0), 2)
                frame = cv2.circle(frame, (x_new, y_new), 3, (0,0,255), -1)

                # --- Accumulate Speed Data ---
                dx = float(x_new - x_old)
                dy = float(y_new - y_old)
                speed = np.sqrt(dx**2 + dy**2) # Speed in pixels/frame

                # Use midpoint for cell location
                mid_x, mid_y = (x_old + x_new) // 2, (y_old + y_new) // 2
                
                cell_c = mid_x // GRID_CELL_SIZE
                cell_r = mid_y // GRID_CELL_SIZE

                if 0 <= cell_r < grid_rows and 0 <= cell_c < grid_cols:
                    speed_sum_grid[cell_r, cell_c] += speed
                    speed_count_grid[cell_r, cell_c] += 1
                # --- End Accumulate Speed Data ---
            
            p0 = good_new.reshape(-1,1,2)
            p0_birth_frames = surviving_birth_frames
        else: # No points survived this frame's tracking
            p0 = np.empty((0,1,2), dtype=np.float32)
            p0_birth_frames = []

        # Overlay the tracks on the frame (ALWAYS do this to show persistent mask)
        img = cv2.add(frame, mask)
        cv2.imshow('Optical Flow Tracking', img)
        
        # Display the updated speed map and get the image
        # Ensure mm_per_pixel is available from the first cell's execution
        # fps is calculated in this cell
        speed_map_image = display_speed_map(speed_sum_grid, speed_count_grid, frame.copy(), GRID_CELL_SIZE, mm_per_pixel, fps)

        # Save the speed map image periodically
        if save_interval_frames > 0 and frame_idx % save_interval_frames == 0:
            timestamp = frame_idx # Or use a more descriptive timestamp if needed
            filename = os.path.join(speed_map_save_path, f"speed_map_frame_{timestamp}.png")
            cv2.imwrite(filename, speed_map_image)
            print(f"Saved speed map to {filename}")


        # Prepare for next iteration (old_gray is updated regardless of points)
        # p0 and p0_birth_frames are already set
    else: # No points to track (len(p0) == 0)
        # Still display frame with potentially old tracks from mask
        img = cv2.add(frame, mask)
        cv2.imshow('Optical Flow Tracking', img)
        
        # Display the current speed map (even if no new points this frame)
        speed_map_image = display_speed_map(speed_sum_grid, speed_count_grid, frame.copy(), GRID_CELL_SIZE, mm_per_pixel, fps)
        
        # Save the speed map image periodically (also if no new points)
        if save_interval_frames > 0 and frame_idx % save_interval_frames == 0:
            timestamp = frame_idx 
            filename = os.path.join(speed_map_save_path, f"speed_map_frame_{timestamp}.png")
            cv2.imwrite(filename, speed_map_image)
            print(f"Saved speed map to {filename}")
        # p0 and p0_birth_frames remain empty

    old_gray = frame_gray.copy()

    # Exit on 'q'
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# --- Final Speed Map Generation ---
# Generate and save the speed map using all accumulated data.
# 'old_frame' should hold the last successfully processed frame, which can be used for overlay.
# If 'old_frame' is not suitable (e.g., if the loop exited before any processing),
# a black frame is a safe fallback.
if 'old_frame' in locals() and old_frame is not None:
    final_overlay_frame = old_frame.copy()
else:
    # Fallback to a black screen if old_frame isn't available (should be rare)
    final_overlay_frame = np.zeros((h, w, 3), dtype=np.uint8) 
    print("Warning: Last processed frame not available for final speed map overlay. Using black background.")

print("\nGenerating final speed map...")
final_speed_map_image = display_speed_map(speed_sum_grid, speed_count_grid, final_overlay_frame, GRID_CELL_SIZE, mm_per_pixel, fps)

# Save the final speed map image
final_filename = os.path.join(speed_map_save_path, "speed_map_final.png")
try:
    cv2.imwrite(final_filename, final_speed_map_image)
    print(f"Saved final speed map to {final_filename}")
except Exception as e:
    print(f"Error saving final speed map: {e}")

# Display the final speed map until a key is pressed
print("Displaying final speed map. Press any key in the 'Speed Map' window to close.")
cv2.imshow('Speed Map', final_speed_map_image) # Ensure this window name matches the one in display_speed_map
cv2.waitKey(0) # Wait indefinitely until a key is pressed
cv2.destroyWindow('Speed Map') # Clean up the specific window
print("Final speed map display closed.")
# --- End Final Speed Map ---


Speed maps will be saved in a folder specific to test number: 6
Using default POSTPROCESS_BASE_DIR: C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess
Speed map directory already exists: C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps_papirtest6
Video trimming enabled for 'papirtest6 - 21 sek.avi': Skipping from frame 630 (0-indexed) up to 882.
Processing will resume at frame 883 (0-indexed).
Saved speed map to C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps_papirtest6\speed_map_frame_300.png
Saved speed map to C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps_papirtest6\speed_map_frame_300.png
Saved speed map to C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps_papirtest6\speed_map_frame_600.png
Saved speed map to C:\Users\HP\OneDrive - NTNU\Desktop\Master\Code\PostProcess\SpeedMaps_papirtest6\speed_map_frame_600.png
Video reader at frame 630 (0-indexed). Jumping to resume at frame 883 (0-inde

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'Speed Map' in function 'cvDestroyWindow'
